## Tutorial of DeepAdapter
### A self-adaptive and versatile tool for eliminating multiple undesirable variations from transcriptome
In this notebook, you will learn how to re-train DeepAdapter with the example dataset.

## 1. Installation and requirements
### 1.1. Installation
To run locally, please open a terminal window and download the code with:
```sh
$ # create a new conda environment
$ conda create -n DA python=3.9
$ # activate environment
$ conda activate DA
$ # Install dependencies
$ pip install deepadapter
$ # Launch jupyter notebook
$ jupyter notebook
```
### 1.2. Download datasets
Please download the open datasets in [Zenodo](https://zenodo.org/records/10494751).
These datasets are collected from literatures to demonstrate multiple unwanted variations, including:
* batch datasets: LINCS-DToxS ([van Hasselt et al. Nature Communications, 2020](https://www.nature.com/articles/s41467-020-18396-7)) and Quartet project ([Yu, Y. et al. Nature Biotechnology, 2023](https://www.nature.com/articles/s41587-023-01867-9)).
* platform datasets: profiles from microarray ([Iorio, F. et al. Cell, 2016](https://www.cell.com/cell/pdf/S0092-8674(16)30746-2.pdf)) and RNA-seq ([Ghandi, M. et al. Nature, 2019](https://www.nature.com/articles/s41586-019-1186-3)).
* purity datasets: profiles from cancer cell lines ([Ghandi, M. et al. Nature, 2019](https://www.nature.com/articles/s41586-019-1186-3)) and tissues ([Weinstein, J.N. et al. Nature genetics, 2013](https://www.nature.com/articles/ng.2764)).

After downloading, place the datasets in the `data/` directory located in the same hierarchy as this tutorial.
* batch datasets: `data/batch_data/`
* platform datasets: `data/platform_data/`
* purity datasets: `data/purity_data/`
  
**Putting datasets in the right directory is important for loading the example datasets successfully.**

To execute a "cell", please press Shift+Enter

## 2. Load the datasets and preprocess
### 2.1. load the modules

In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
import pandas as pd
import numpy as np

from deepadapter.utils import data_utils as DT
from deepadapter.finetune_utils as FTUT
from deepadapter.params import dl_finetune_params as DLPARAM

/Users/mengjizhang/miniconda3/envs/DA/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 2.2. Load the demonstrated datasets
We ultilize Batch-LINCS for demonstration. To load datasets of platform and purity variations, please download them in Zenodo (https://zenodo.org/records/10494751).
  * In the tutorial, we have **data** for gene expression, **batches** for unwanted variations, and **donors** for biological signals.
  * If you want to fine-tune with your own data, please refer to `DeepAdapter-YourOwnData-Finetune.ipynb`.

In [2]:
loadTransData = DT.LoadTransData()
data, batches, wells, donors, infos, test_infos = loadTransData.load_lincs_lds1593()
ids = np.arange(len(data))

Load LINCS LDS 1593, size of (709, 10112)


### 2.3 Load the genes used in pre-trained model
Before fine-tuning, make sure that the loaded genes are the genes used in the pre-trained model. The pre-trained models can be found in the folder of `models`.

In [ ]:
load_dir = "models/pretrained_LINCS_Quartet/"
pretrain_genes = pd.read_csv(os.path.join(load_dir, "gene.csv"))["gene"]
try:
	data = data[pretrain_genes]
except Exception as e:
    raise("Inconsistent gene set between this dataset and pretrained dataset")

### 2.3. Preprocess the transcriptomic data
The gene expression profiles are preprocessed by sample normalization, gene ranking, and log normalization. Let $S_i = \sum_l x_{i l}$ denote the sum over all genes. In sample normalization, we divide $S_i$ for every sample and multiply a constant 10000 ([Xiaokang Yu et al. Nature communications, 2023](https://www.nature.com/articles/s41467-023-36635-5)):
$$x_{i l} = \frac{x_{i l}}{S_i} 10^4.$$
Then, we sort genes by their expression levels and perform the log transformation $x_{i l} = \log {(x_{i l} + 1)}$.

In [ ]:
prepTransData = DT.PrepTransData()
raw_df = prepTransData.sample_norm(data)
input_arr = prepTransData.sample_log(raw_df)
bat2label, label2bat, unwanted_labels, unwanted_onehot = prepTransData.label2onehot(batches)

## 3. Fine-tune DeepAdapter
### 3.1. Adjust DeepAdapter's parameters
The parameters for DeepAdapter are as follows (**Note: you can revise parameter directly in `net_args`, e.g., `net_args.batch_size = 512`.**):
* **epochs**: the fine-tune epochs of DeepAdapter, default = $5000$
* **ae_epochs**: the warmup epochs of autoencoder in DeepAdapter, default = $400$
* **batch_epochs**: the warmup epochs of discriminator in DeepAdapter, default = $50$
* **batch_size**: the batch size of dataloader, default = $256$
* **hidden_dim**: the hidden units of autoencoder in DeepAdapter, default = $256$
* **z_dim**: the latent units of autoencoder in DeepAdapter, default = $128$
* **drop**: the dropout rate of DeepAdapter, default = $0.3$
* **lr_lower_ae**: the lower learning rate of autoencoder in DeepAdapter, default = $1e-5$
* **lr_upper_ae**: the upper learning rate of autoencoder in DeepAdapter, default = $5e-4$
* **lr_lower_batch**: the lower learning rate of discriminator in DeepAdapter, default = $1e-5$
* **lr_upper_batch**: the upper learning rate of discriminator in DeepAdapter, default = $5e-4$

In [ ]:
net_args = DLPARAM.load_dl_params()

### 3.2. Split dataset
* For the tutorial, we extract the biosamples across all batches as the test set; then split the rest into training and validation set randomly.</br>
That means the training data seen by DeepAdapter doesn't disperse across all unwanted variations while the testing data does.</br>
Acutally, this split method could increase the training difficulty.
* For your own dataset, you can split the dataset randomly using the function `DT.data_split_random`.</br>
Please refer to `DeepAdapter-YourOwnData-Tutorial.ipynb`.

In [ ]:
train_data, train_labels, train_labels_hot, \
    val_data, val_labels, val_labels_hot, \
    test_data, test_labels, test_labels_hot, \
    train_ids, val_ids, test_ids, \
    tot_train_val_idxs, tot_train_idxs, tot_val_idxs, tot_test_idxs = DT.data_split_lds1593(input_arr, unwanted_labels, unwanted_onehot, ids, infos, test_infos)

In [ ]:
train_bios, val_bios, test_bios = donors[tot_train_idxs], donors[tot_val_idxs], donors[tot_test_idxs]

In [ ]:
bio_label2bat = {t:t for t in set(train_bios)}

### 3.3. Fine-tune DeepAdapter
Given that the fine-tuned dataset encompasses a different number of batch categories (4 batches) compared to the pre-trained dataset (21 batches), we modify the last layer of the discriminatory network to classify 4 batch categories instead of 21. We unfreeze all layers and train the fine-tuned models for 5K epochs. This procedure is repeated 100 times, with performances assessed by an independent testing set of 24 samples.

In [ ]:
ft_num = net_args.ft_num

out_dir = f"models/finetune_LINCS_Quartet/record_{ft_num}"
os.makedirs(out_dir, exist_ok = True)

num_platform = len(bat2label)
data, labels, labels_hot, ids = unwanted_onehot, unwanted_labels, unwanted_onehot, np.arange(len(unwanted_onehot))

FTUT.test_finetune(data, labels, labels_hot, donors, ids, label2bat, load_dir, out_dir, net_args, num_platform, finetune_num = ft_num, n_test = 100)